In [158]:
import csv
import datetime
import time
import re
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
import pandas as pd
import translators as ts
pd.options.mode.chained_assignment = None
csv.field_size_limit(2147483647)

2147483647

In [159]:
# Original dataset
df = pd.read_csv("data/original/designs.csv", sep="‡", engine="python", index_col="id")
df

,user_id,name,short_description,long_description,link,video,credits,category,country_id,award_year,...,bmc_partners,bmc_activities,bmc_resources,bmc_value_propositions,bmc_customer_relationships,bmc_channels,bmc_customer_segments,bmc_cost_structure,bmc_revenue_streams,self
id,,,,,,,,,,,,,,,,,,,,,
1,1,Scanadu,A scanner packed with sensors that enables any...,"As if it is reading your mind, the Scanadu Sco...",https://www.scanadu.com/scout/,"<iframe width=""560"" height=""315"" src=""//www.yo...",Yves Béhar,1.0,203.0,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
10,4,Oculus Rift,Oculus Rift in an effort to revolutionize the ...,Virtual Reality have been talked about for yea...,http://www.oculus.com,"<iframe width=""640"" height=""360"" src=""//www.yo...","Palmer Luckey: Oculus VR® & © 2014 Oculus VR, ...",4.0,203.0,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16,4,WRISTIFY,A thermoelectric bracelet that regulates the t...,Wristify by Embr Labs uses all-natural waves o...,http://www.embrlabs.com,"<iframe width=""560"" height=""315"" src=""//www.yo...","Embr labs: Matthew Smith, Sam Shames, Michael...",1.0,203.0,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17,4,Liftware,Tremor cancellation device thats fits into the...,The Liftware system is designed for people who...,http://www.liftlabsdesign.com,"<iframe width=""560"" height=""315"" src=""//www.yo...","Lift Labs - Lynx Design, Inc.",1.0,203.0,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
18,4,Sproutling,"The world's first sensing, learning, predictin...","A wearable band for your baby, a smart charger...",http://www.sproutling.com,"<iframe width=""560"" height=""315"" src=""//www.yo...",Sproutling,1.0,203.0,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7738,6696,Algae-powered photovoltaic cell,Using algae to power a microprocessor with not...,NaN,https://www.cam.ac.uk/research/news/scientists...,NaN,P. Bombelli et al,3.0,128.0,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7739,6696,Elonroad,A holistic charging solution for all electric ...,NaN,https://elonroad.com,https://www.youtube.com/embed/lH1QdxezKoY,Elonroad,5.0,127.0,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7740,6680,Critter Cuts,"To encourage families so they can prepare, coo...",NaN,https://kellyrahardja.com/Critter-Cuts,NaN,Kelly Rahardja,4.0,80.0,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### Adding goals

In [160]:
# Design goals (pivot table)
dg = pd.read_csv("data/original/design-goals.csv", sep="‡", engine="python", index_col="id")
dg.tail()

,design_id,goal_id
id,,
11667,7740,25
11668,7740,5
11669,7741,8
11670,7741,12
11671,7742,3


In [161]:
# Adding goals from the pivot table to the original dataset
df['goals'] = ""
for dg_row in dg.values:
    if dg_row[0] in df.index:
        goal_list = df.at[dg_row[0], 'goals'].split()
        goal_list.append(dg_row[1])
        df.at[dg_row[0], 'goals'] = ' '.join(map(str, goal_list))
df.loc[:, 'goals'].tail()

id
7738       7 9
7739       7 9
7740    4 25 5
7741      8 12
7742         3
Name: goals, dtype: object

### Filtering drafts

In [162]:
# Filtering out drafts
print(f"Number of drafts to remove: {sum(df['status'] == 'draft')}")
df = df.loc[df['status'] != 'draft']

Number of drafts to remove: 13


### Merging descriptions

In [163]:
# Merging "how/why/dtil_description" into "long_description"
print(f"Number of descriptions to merge: {sum(df['long_description'].isnull())}")
df['combined_description'] = df['how_description'] + ' ' + df['why_description'] + ' ' + df['dtil_description']
df.long_description.fillna(df.combined_description, inplace=True)
df.drop(['dtil_description', 'how_description', 'why_description', 'combined_description'], axis=1, inplace=True)

Number of descriptions to merge: 619


### Converting column datatypes

In [164]:
# Converting category column type to string
df = df.astype({'category': 'Int64'}).astype({'category': 'string'})
df['category'].head()

id
1     1
10    4
16    1
17    1
18    1
Name: category, dtype: string

### Web scraping

In [165]:
# Initiates the browser for web scraping
def init_browser():
    opts = Options()
    opts.headless = True
    return Firefox(options=opts)

In [166]:
# Removes any remaining html tags (most should be removed by the scraper already)
def clean_text(txt):
    return " ".join(re.sub(r'<.*?>', ' ', txt).split())

In [167]:
# Scraping the title, body and metadata from a page
def scrape(url, browser, sleep=0):
    time.sleep(sleep)
    page_title = page_body = meta_title = meta_description = ""
    try:
        browser.get(url)
        time.sleep(sleep)
    except:
        browser.delete_all_cookies()
        return ("", "", "", "")
    try:
        page_title = clean_text(browser.title)
    except:
        page_title = ""
    try:
        page_body = clean_text(browser.find_element("tag name", "body").text)
    except:
        page_body = ""
    try:
        meta_title = clean_text(browser.find_element(
            "css selector", "meta[property='og:title']").get_attribute("content"))
    except:
        meta_title = ""
    try:
        meta_description = clean_text(browser.find_element(
            "css selector", "meta[property='og:description']").get_attribute("content"))
    except:
        meta_description = ""
    return (page_title, page_body, meta_title, meta_description)

In [122]:
# Scraping (can take ~11h for the full dataset)
links = df["link"]
ff = init_browser()
for id, link in links.items():
    p_t = p_b = m_t = m_d = ""
    try:
        p_t, p_b, m_t, m_d = scrape(link, ff)
        print(id, link, p_t)
    except:
        print("reattempting link: ", link)
        ff = init_browser()
        try:
            p_t, p_b, m_t, m_d = scrape(link, ff, 5)
            print(id, link, p_t)
        except:
            print("failed, resetting browser: ", link)
            ff = init_browser()
    df.loc[id, "page_title"] = p_t
    df.loc[id, "page_body"] = p_b
    df.loc[id, "meta_title"] = m_t
    df.loc[id, "meta_description"] = m_d

5522 http://blog.graphisoft.lat/centro-comunitario-la-perseverancia/ Centro Comunitario “La Perseverancia” – GRAPHISOFT Latinoamérica
5523 https://www.unocup.com Unocup
5524 https://www.runwithrun.com/run-projects/badgood RUN
5525 https://www.oui.us 
5526 https://www.mylo-unleather.com/ Mylo™ Unleather | Sustainable Vegan Mycelium Leather
5527 https://www.tomtex.co/ TÔMTEX
5528 https://goclove.com Best Shoes For Healthcare Providers - Nurses - Hospitals 2021 | Clove
5529 https://slowfactory.foundation/open-education Open Education
5530 http://www.fernandolaposse.com/projects/totomoxtle/ Totomoxtle |
5531 https://www.behance.net/gallery/103352151/Bukas-Material-Messages-for-OFWs-and-their-families Behance
5532 https://sumjaileung.myportfolio.com/polyu-eid-guerrilla-planter Adobe Portfolio | Build your own personalized website
5533 https://www.applemok.com/culturalrun-2020 Error
5534 https://givepower.org/introducing-first-givepower-solar-water-farm/ Introducing The First GivePower Solar

### Translating

In [169]:
# Translating a string in chunks of n words (to work around the length limit of translation requests)
def translate(str, lang='es', n=200):
    result = ""
    try:
        result = ts.google(str, lang)
    except:
        try:
            arr = str.split()
            result = ' '.join([ts.google(txt, lang) for txt in [' '.join(arr[i:i+n]) for i in range(0, len(arr), n)]])
        except:
            result = "translation error"
    return result

In [170]:
# Selecting projects from Diseño Responde (these have text written in Spanish)
dr = df['competition_id'] != 0
df.loc[dr, 'long_description'].head()

id
5522    Este diseño es el resultado del trabajo multid...
5545    Para solucionar esta situación, yo propuse 5 p...
5546    NODO es un servicio y plataforma de comunicaci...
5551    El diseño se basa en el concepto de castillo m...
5552    La situación ha generado problemas en todos lo...
Name: long_description, dtype: object

In [177]:
# Translating from Spanish to English
print(f"Number of projects to translate: {len(df[dr])}")
cols_translate = ['name', 'long_description', 'page_title', 'page_body', 'meta_title', 'meta_description']
for col in cols_translate:
    df.loc[dr, col] = df.loc[dr, col].map(lambda x: translate(x))

Number of projects to translate: 399


In [178]:
# Descriptions after translation
df.loc[dr, 'long_description'].head()

id
5522    This design is the result of multidisciplinary...
5545    To solve this situation, I proposed 5 graphic ...
5546    Node is a communication and notification servi...
5551    The design is based on the concept of Medieval...
5552    The situation has generated problems in all as...
Name: long_description, dtype: object

### Removing rarely used goals

In [179]:
# Removing a goal from the dataset
def removeGoal(c):
    df.loc[df['goals'].str.contains(f"{c}"), "goals"] = df.loc[df['goals'].str.contains(f"{c}"), "goals"].map(lambda x: x.replace(f"{c}", "").strip())

In [180]:
# Checking the count of each goal being used
for g in range(1, 26):
    print(f"{g}: {sum(df['goals'].str.contains(f'{g}'))}")

1: 3921
2: 2165
3: 2697
4: 807
5: 694
6: 493
7: 584
8: 582
9: 1233
10: 465
11: 1025
12: 1388
13: 567
14: 189
15: 471
16: 231
17: 175
18: 109
19: 289
20: 392
21: 148
22: 156
23: 7
24: 2
25: 16


In [181]:
# Removing goals that are barely used (otherwise, the validation set might contain goals that aren't in the training set)
removeGoal(23)
removeGoal(24)
removeGoal(25)

In [182]:
# Removing designs with no goals left
ids = df.loc[df['goals'] == ""].index
df = df.drop(index=ids)
ids

Int64Index([7263, 7542], dtype='int64', name='id')

### Marking the training, validation and test sets

In [183]:
# Checking the number of rows for splitting
total = len(df)
tenth = round(len(df)*0.1)
f"total number of rows: {total} (10% is {tenth})"

'total number of rows: 7460 (10% is 746)'

In [184]:
# Setting initial validation values
df["is_valid"] = 0

In [185]:
# Training set: first 80%
f"Number of projects in the training set: {len(df.iloc[0:total-tenth*2])}"

'Number of projects in the training set: 5968'

In [186]:
# Validation set: middle 10%
df.iloc[total-tenth*2:total-tenth]["is_valid"] = 1
f"Number of projects in the training set: {sum(df['is_valid'] == 1)}"

'Number of projects in the training set: 746'

In [187]:
# Test set: last 10%
df.iloc[total-tenth:total]["is_valid"] = 2
f"Number of projects in the test set: {sum(df['is_valid'] == 2)}"

'Number of projects in the test set: 746'

### Setting trust

In [188]:
# Checking whether the project was nominated by someone from The Index Project
indexUserIds = pd.read_csv("data/original/index-ids.csv", engine="python", index_col="id").index
df["tip"] = df["user_id"].map(lambda x: x in indexUserIds).astype(int)
f"Number of projects nominated by members of The Index Project: {sum(df['tip'])}"

'Number of projects nominated by members of The Index Project: 3239'

In [189]:
# Marking projects from Diseno Responde as self nominated ones 
# (the attribute didn't exist when the competition took place, and all nominees from there were self-nominated projects)
df.loc[df["competition_id"] == 1, "self"] = 1
f"Number of projects that were self-nominated: {sum(df['self'])}"

'Number of projects that were self-nominated: 420'

In [190]:
# Setting the "time" attribute based on relative creation time
train_set = df["is_valid"] == 0
# note: created_at had to be substituted by id as a proxy, since the created_at column contains many errors
# old line: df.loc[train_set, "time"] = df.loc[train_set, "created_at"].map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timestamp())
df.loc[train_set, "time"] = df.loc[train_set].index
minTime = df.loc[train_set, "time"].min()
maxTime = df.loc[train_set, "time"].max()
diffTime = maxTime - minTime
df.loc[train_set, "time"] = df.loc[train_set, "time"].map(lambda x: (x - minTime) / diffTime)
df.loc[train_set, "time"]

id
1       0.000000
10      0.001454
16      0.002424
17      0.002586
18      0.002747
          ...   
6185    0.999354
6186    0.999515
6187    0.999677
6188    0.999838
6189    1.000000
Name: time, Length: 5968, dtype: float64

In [191]:
# Setting the "trust" attribute
df.loc[train_set, "trust"] = df.loc[train_set].apply(lambda x: 0.8 * x["time"] + 0.2 * max(x["self"], x["tip"]), axis=1)
df.loc[train_set, "trust"]

id
1       0.200000
10      0.201164
16      0.201939
17      0.202069
18      0.202198
          ...   
6185    0.999483
6186    0.999612
6187    0.999741
6188    0.799871
6189    0.800000
Name: trust, Length: 5968, dtype: float64

### Selecting specific columns

In [192]:
# Selecting specific columns to keep
cols = ['name', 'long_description', 'page_title', 'page_body', 'meta_title', 'meta_description', 'category', 'goals', 'trust', 'is_valid']
df = df.loc[:, cols]
df.head()

,name,long_description,page_title,page_body,meta_title,meta_description,category,goals,trust,is_valid
id,,,,,,,,,,
1,Scanadu,"As if it is reading your mind, the Scanadu Sco...",Scanadu Scout™ - Scanadu.com,Skip to content Scanadu Scout™ Reviews Neuigke...,Scanadu Scout™ - Scanadu.com,A scanner packed with sensors that enables any...,1,3,0.200000,0
10,Oculus Rift,Virtual Reality have been talked about for yea...,"Oculus VR headsets, games and equipment – Meta...",NaN,NaN,NaN,4,17,0.201164,0
16,WRISTIFY,Wristify by Embr Labs uses all-natural waves o...,Personal Cooling Device for Menopause Hot Flas...,Skip to content End of Summer Sale | Use code ...,Personal Cooling Device for Menopause Hot Flas...,"Founded at MIT, the Embr Wave® wristband quick...",1,3,0.201939,0
17,Liftware,The Liftware system is designed for people who...,Liftware - Eat with confidence,It looks like you're accessing this page from ...,Liftware - Eat with confidence,Liftware's selection of stabilizing and leveli...,1,3,0.202069,0
18,Sproutling,"A wearable band for your baby, a smart charger...",NaN,NaN,NaN,NaN,1,3,0.202198,0


### Saving to files

In [198]:
# Saving training and testing sets
df.loc[df["is_valid"] <= 1].to_csv('designs-train.csv', sep="‡")
df.loc[df["is_valid"] == 2].to_csv('designs-test.csv', sep="‡")